# nuImages devkit tutorial

Welcome to the nuImages tutorial.
This demo assumes the database itself is available at `/data/sets/nuimages`, and loads a mini version of the dataset.

In [3]:
#run once
#!mkdir -p /data/sets/nuimages  # Make the directory to store the nuImages dataset in.
#!wget https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz  # Download the nuImages mini split.

#!tar -xf nuimages-v1.0-mini.tgz -C /data/sets/nuimages  # Uncompress the nuImages mini split.
#!pip install nuscenes-devkit &> /dev/null  # Install nuImages.

mkdir: cannot create directory ‘/data’: Permission denied
--2024-04-23 18:35:16--  https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 2600:9000:21c7:bc00:1b:5ef:6040:93a1, 2600:9000:21c7:2a00:1b:5ef:6040:93a1, 2600:9000:21c7:3200:1b:5ef:6040:93a1, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|2600:9000:21c7:bc00:1b:5ef:6040:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117929607 (112M) [application/x-tar]
Saving to: ‘nuimages-v1.0-mini.tgz’

nuimages-v1.0-mini. 100%[===================>] 112,47M  84,9MB/s    in 1,3s    

2024-04-23 18:35:17 (84,9 MB/s) - ‘nuimages-v1.0-mini.tgz’ saved [117929607/117929607]

tar: /data/sets/nuimages: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


## Initialization
To initialize the dataset class, we run the code below. We can change the dataroot parameter if the dataset is installed in a different folder. We can also omit it to use the default setup. These will be useful further below.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from nuimages import NuImages

In [3]:
def bbox2yolo(sizee, boxx):
    dw = 1. / sizee[0]
    dh = 1. / sizee[1]
    x_center = (boxx[0] + boxx[1]) / 2.0
    y_center = (boxx[2] + boxx[3]) / 2.0
    width = boxx[1] - boxx[0]
    height = boxx[3] - boxx[2]
    
    x_center = x_center * dw
    width = width * dw
    y_center = y_center * dh
    height = height * dh


    
    
    
    return (x_center, y_center, width, height)

#now get the 'category_name' and assign a number instead 
classes = ['barrier', 'bicycle', 'bus', 'car', 'construction_vehicle', 'motorcycle', 'pedestrian', 'traffic_cone', 'trailer', 'truck']
#make a dictionary where each class got a number
class_dict = {classes[i]: i for i in range(len(classes))}
print(class_dict)

{'barrier': 0, 'bicycle': 1, 'bus': 2, 'car': 3, 'construction_vehicle': 4, 'motorcycle': 5, 'pedestrian': 6, 'traffic_cone': 7, 'trailer': 8, 'truck': 9}


In [6]:
import numpy as np
import torch
import torchvision.transforms as T
import concurrent.futures
import os


def process_sample(i):
    sample = nuim.sample[i]
    key_camera_token = sample['key_camera_token']
    
    # Render image using PyTorch
    im, obj = nuim.render_image_2(
        key_camera_token,
        annotation_type='objects',
        with_category=True,
        with_attributes=True,
        box_line_width=-1,
        render_scale=5
    )
    
    w, h = im.size
    
    # Convert image to PyTorch tensor
    transform = T.ToTensor()
    im_tensor = transform(im).unsqueeze(0).cuda()
    
    labels_data = []
    
    for obj_info in obj:
        xmin, ymin, xmax, ymax = obj_info['bbox']
        
        # Convert bbox to YOLO format
        bb = bbox2yolo((w, h), (xmin, xmax, ymin, ymax))
        
        category_name = obj_info['category_name']
        if 'vehicle.construction' in category_name:
            labels_data.append(f"{class_dict['construction_vehicle']} {' '.join(map(str, bb))}\n")
            print('cons vehicle')
        if 'movable_object.trafficcone' in category_name:
            labels_data.append(f"{class_dict['traffic_cone']} {' '.join(map(str, bb))}\n")
            print('trafficcone')
        for class_name in classes:
            if class_name in category_name:
                labels_data.append(f"{class_dict[class_name]} {' '.join(map(str, bb))}\n")
                break
                
    with open(os.path.join(labels_dir, f"{i}.txt"), 'w') as f:
        f.writelines(labels_data)
    
    im.save(os.path.join(images_dir, f"{i}.png"))

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

amount_samples = 8721810 # change to amount of samples in data, or just way to much and run until error


In [ ]:
nuim = NuImages(dataroot='/home/robolab/Downloads/nuimages-v1.0-all-metadata', version='v1.0-val', verbose=True, lazy=True)
labels_dir = '/home/robolab/Downloads/yolov9/val2/labels' #for the val
images_dir = '/home/robolab/Downloads/yolov9/val2/images'  

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

# Parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_sample, range(amount_samples)) 

In [ ]:
nuim = NuImages(dataroot='/home/robolab/Downloads/nuimages-v1.0-all-metadata', version='v1.0-test', verbose=True, lazy=True)
labels_dir = '/home/robolab/Downloads/yolov9/test2/labels' #for the val
images_dir = '/home/robolab/Downloads/yolov9/test2/images'  

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

# Parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_sample, range(amount_samples)) 

Loading nuImages tables for version v1.0-test...
Done loading in 0.000 seconds (lazy=True).
Loaded 9752 sample(s) in 0.020s,
Loaded 9752 sample(s) in 0.030s,
Loaded 9752 sample(s) in 0.038s,
Loaded 9752 sample(s) in 0.049s,
Loaded 9752 sample(s) in 0.073s,
Loaded 9752 sample(s) in 0.131s,
Loaded 9752 sample(s) in 0.173s,
Loaded 9752 sample(s) in 0.182s,
Loaded 9752 sample(s) in 0.489s,
Loaded 9752 sample(s) in 0.510s,
Loaded 9752 sample(s) in 0.546s,
Loaded 9752 sample(s) in 0.545s,
Loaded 9752 sample(s) in 0.591s,
Loaded 126276 sample_data(s) in 0.863s,
Loaded 126276 sample_data(s) in 1.499s,
Loaded 126276 sample_data(s) in 1.573s,
Loaded 126276 sample_data(s) in 1.438s,
Loaded 126276 sample_data(s) in 1.811s,
Loaded 126276 sample_data(s) in 2.253s,
Loaded 126276 sample_data(s) in 3.265s,
Loaded 126276 sample_data(s) in 4.006s,
Loaded 126276 sample_data(s) in 4.068s,
Loaded 126276 sample_data(s) in 4.543s,
Loaded 126276 sample_data(s) in 4.701s,
Loaded 126276 sample_data(s) in 4.628s,

In [ ]:
nuim = NuImages(dataroot='/home/robolab/Downloads/nuimages-v1.0-all-metadata', version='v1.0-train', verbose=True, lazy=True)
labels_dir = '/home/robolab/Downloads/yolov9/train2/labels' #for the val
images_dir = '/home/robolab/Downloads/yolov9/train2/images'  
if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
if not os.path.exists(images_dir):
    os.makedirs(images_dir)


# Parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_sample, range(amount_samples)) 